<a href="https://colab.research.google.com/github/PETEPEtrek/EPLAnalysis/blob/main/EPLTeamsAnalysisProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is my EPL(English Premier League) analysis.

In [ ]:
# Loading packages
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

In [ ]:
# estimators and utilities
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.neural_network import MLPClassifier
import matplotlib.ticker as mtick 
# preprocess & engineering
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer, OneHotEncoder, PolynomialFeatures
# model scoring and validation
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
# calibration of probabilities
from sklearn.calibration import  CalibratedClassifierCV, calibration_curve

#**Loading Data**

In [ ]:
# Путь к папке с заданием/"
# Путь к файлу с модулями
path_to_modules_notebook = "drive/My Drive/Colab Notebooks/results.csv"

# В Google Colab подключаемся к диску
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')
    path_to_modules_notebook = path_to_modules_notebook

Mounted at /content/drive


In [ ]:
# Uploading data as a .csv file 
epl_df = pd.read_csv(path_to_modules_notebook, index_col=0,encoding='unicode_escape')
epl_df.head()

,DateTime,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR
Season,,,,,,,,,,,,,,,,,,,,,
1993-94,1993-08-14T00:00:00Z,Arsenal,Coventry,0,3,A,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-94,1993-08-14T00:00:00Z,Aston Villa,QPR,4,1,H,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-94,1993-08-14T00:00:00Z,Chelsea,Blackburn,1,2,A,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-94,1993-08-14T00:00:00Z,Liverpool,Sheffield Weds,2,0,H,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-94,1993-08-14T00:00:00Z,Man City,Leeds,1,1,D,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# **Column	Description**
* Season:	Match Season
* DateTime:	Match Date and Time (yyyy-mm-dd hh:mm:ss)
* HomeTeam:	Home Team
* AwayTeam:	Away Team
* FTHG:	Full Time Home Team Goals
* FTAG:	Full Time Away Team Goals
* FTR:	Full Time Result (H=Home Win, D=Draw, A=Away Win)
* HTHG:	Half Time Home Team Goals
* HTAG:	Half Time Away Team Goals
* HTR:	Half Time Result (H=Home Win, D=Draw, A=Away Win)
* Referee:	Match Referee
* HS:	Home Team Shots
* AS:	Away Team Shots
* HST:	Home Team Shots on Target
* AST:	Away Team Shots on Target
* HC:	Home Team Corners
* AC:	Away Team Corners
* HF:	Home Team Fouls Committed
* AF:	Away Team Fouls Committed
* HY:	Home Team Yellow Cards
* AY:	Away Team Yellow Cards
* HR:	Home Team Red Cards
* AR:	Away Team Red Cards



As we can see, there are many NaN values which could be due to the fact that this information wasn't tracked. Let's see last year when there was a NaN value

In [ ]:
epl_df[epl_df.isna().any(axis=1)]

,DateTime,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR
Season,,,,,,,,,,,,,,,,,,,,,
1993-94,1993-08-14T00:00:00Z,Arsenal,Coventry,0,3,A,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-94,1993-08-14T00:00:00Z,Aston Villa,QPR,4,1,H,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-94,1993-08-14T00:00:00Z,Chelsea,Blackburn,1,2,A,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-94,1993-08-14T00:00:00Z,Liverpool,Sheffield Weds,2,0,H,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-94,1993-08-14T00:00:00Z,Man City,Leeds,1,1,D,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999-00,2000-05-14T00:00:00Z,Sheffield Weds,Leicester,4,0,H,2.0,0.0,H,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-00,2000-05-14T00:00:00Z,Southampton,Wimbledon,2,0,H,0.0,0.0,D,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-00,2000-05-14T00:00:00Z,Tottenham,Sunderland,3,1,H,1.0,1.0,D,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


So, as we could see, they probably didn't trace such information as Referee and cards.

Let's check some general information about dataset

In [ ]:
epl_df.describe()

,FTHG,FTAG,HTHG,HTAG,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR
count,11113.000000,11113.000000,10189.000000,10189.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000,8289.000000
mean,1.518222,1.141456,0.684758,0.501031,13.503921,10.644589,6.117264,4.768247,6.081795,4.784292,11.379057,11.873447,1.415852,1.746532,0.062854,0.089396
std,1.301981,1.135907,0.836299,0.722299,5.258636,4.583010,3.327521,2.792965,3.095070,2.728954,3.784956,3.957427,1.189134,1.267468,0.253416,0.297341
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,10.000000,7.000000,4.000000,3.000000,4.000000,3.000000,9.000000,9.000000,1.000000,1.000000,0.000000,0.000000
50%,1.000000,1.000000,0.000000,0.000000,13.000000,10.000000,6.000000,4.000000,6.000000,4.000000,11.000000,12.000000,1.000000,2.000000,0.000000,0.000000
75%,2.000000,2.000000,1.000000,1.000000,17.000000,13.000000,8.000000,6.000000,8.000000,6.000000,14.000000,14.000000,2.000000,3.000000,0.000000,0.000000
max,9.000000,9.000000,5.000000,5.000000,43.000000,30.000000,24.000000,20.000000,20.000000,19.000000,33.000000,29.000000,7.000000,9.000000,3.000000,2.000000


As we can see, there are maximum 9 goals were scored both by home and away team. Also average number of goals is 1.5 for home team and 1.14 for away team which could mean that it easier to score at your own stadium rather than rival's stadium.

# Questions

Let's find an answer for some questions:


*   How yellow and red cards affects on command' results
*   How good or bad team' results for each season


For some questions, we would only check the TOP-6 clubs: Arsenal, Manchester United, Manchester City, Liverpool, Chelsea and Tottanham



# EDA
Let's add some new features. Firstly, we will add the result points number at the end for each season. As we know, victory is estimates as 3 points, draw as 1, and defeat as 0 points.

In [ ]:
for team in ['HomeTeam', 'AwayTeam']:
  